Data augmentation

In [ ]:
import numpy as np
from skimage import io
import os
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from skimage import io

# Construct an instance of the ImageDataGenerator class
# Pass the augmentation parameters through the constructor. 

datagen = ImageDataGenerator(
        rotation_range=45,     #Random rotation between 0 and 45
        width_shift_range=0.5,   #% shift
        height_shift_range=0.5,
        shear_range=0.5,
        zoom_range=0.4,
        horizontal_flip=True,
        vertical_flip=True,

        fill_mode='nearest'
        ) 

image_directory = '/content/drive/MyDrive/Data/skin_data/ActualData/train/notmelanoma/'
SIZE = 320
dataset = []

my_images = os.listdir(image_directory)
for i, image_name in enumerate(my_images):
    if (image_name.split('.')[1] == 'jpg'):
        image = io.imread(image_directory + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE,SIZE))
        dataset.append(np.array(image))
#         print('image')
x = np.array(dataset)

In [ ]:
i = 0
for batch in datagen.flow(x, batch_size=64,  
                          save_to_dir='/content/drive/MyDrive/Data/skin_data/Augmented/Train/notmelanoma', 
                          save_prefix='aug', 
                          save_format='jpg'):
    i += 1
    if i > 40:
        break


Convolutional Neural Network

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense,LeakyReLU,MaxPool2D
from keras.layers import Conv2D,MaxPooling2D
from keras import backend as k
from keras.preprocessing import image
import numpy as np
from tensorflow.keras.optimizers import Adam,RMSprop
import cv2
import os
from tensorflow.keras import regularizers
train_path='/content/drive/MyDrive/Data/skin_data/Augmented/Train'
test_path = '/content/drive/MyDrive/Data/skin_data/Augmented/Val'
target_size=(150,150)


In [ ]:
train = ImageDataGenerator(
    
    width_shift_range= 0.2, height_shift_range= 0.2,
    rotation_range= 90, rescale = 1/255,
    horizontal_flip= True, vertical_flip=True,
)

validation = ImageDataGenerator(
    rescale = 1/255,
    horizontal_flip = True,


)
train_dataset = train.flow_from_directory(
    train_path,
    target_size,
    batch_size = 32,
    color_mode="rgb",
    class_mode = 'binary'
     
 )
test_dataset = validation.flow_from_directory(
    test_path,
    target_size,
    batch_size = 32,
    color_mode="rgb",
    class_mode = 'binary'
)

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

opt =Adam(lr=0.0001)

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics =['accuracy'] )


In [ ]:
history = model.fit(
    train_dataset,
    steps_per_epoch = 6,
    epochs = 50,
    validation_data = test_dataset,
    validation_steps = 32,
    # callbacks=[early_stopping]
)

In [ ]:
import os
y_actual = []
y_test = []
valMelnoma = '/content/drive/MyDrive/Data/skin_data/Augmented/Val/melanoma/'
valNot = '/content/drive/MyDrive/Data/skin_data/Augmented/Val/notmelanoma/'

for i in os.listdir(valMelnoma):
  rslt =[]
  img = image.load_img(valMelnoma+'/'+i,target_size=(150,150))
  img = image.img_to_array(img)
  img = np.expand_dims(img,axis=0)
  ##############
  rslt = model.predict(img)
  rslt = np.round(rslt)
  rslt = np.array(rslt).astype(int)
  y_test.append(rslt[0,0])

  y_actual.append(0)
  
  
for i in os.listdir(valNot):
  rslt =[]
  img = image.load_img(valNot+"/"+i,target_size=(150,150))
  img = image.img_to_array(img)
  img = np.expand_dims(img,axis=0)
  ##############
  rslt = model.predict(img)
  rslt = np.round(rslt)
  rslt = np.array(rslt).astype(int)
  y_test.append(rslt[0,0])
  y_actual.append(1)


In [ ]:
y_actual = np.array(y_actual).astype(int)
y_test = np.array(y_test).astype(int)

y_test


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_actual) 
cm


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
      print('Confusion matrix, without normalization')
      print(cm)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat = cm, figsize = ( 6 , 6 ), cmap = plt.cm.Blues)
plt.xlabel( 'Predictions' , fontsize = 18 ) 
plt.ylabel( 'Actuals' , fontsize = 18 )
plt.title( 'Confusion Matrix' , fontsize = 18 ) 
plt.show()


In [ ]:
from sklearn.metrics import  classification_report
print(classification_report(y_test,y_actual))